<h1><span style="color:purple">Transfer Learning with TensorFlow</span> </h1>

![](https://images.unsplash.com/photo-1496483648148-47c686dc86a8?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1052&q=80)

<h2><span style="color:Purple">Introduction</span> </h2>

Classical methods showed poor performance for image and text classification problems. The models which based on deep learning used to solve these problems. There are very much popular models which tested and open source that they are avaible for transfer learning.
The models such as ResNet are prebuilted and very complicated structures. TensorFlow Hub proposed a variety of ML models. You can use these model writting a single line of code.
In this tutorial, I am going to show how to use models pretrained in TensorFlow Hub. If you want to install Tensorflow Hub, you can use `pip install --upgrade tensorflow_hub` command. After installing, first of all, to use I am going to import this library `import tensorflow_hub as hub` command.

<h2><span style="color:Purple">Using the model in TensorFlow Hub</span> </h2>

For example, you can use a pretrained text embedding model in tensorFlow Hub such as `model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")`
Token in this model based text embedding trained on English Google News 200B corpus. Text embedding is a multidimensional vector of numeric representation. Note that you can onlu load and use it to get a result with your own data. 

You can choose the model in [TensorFlow Hub](https://tfhub.dev/) according to your the dataset. There are very much model for image, text, video, and audio datasets.

<h2><span style="color:Purple">Image Classification by Transfer Learning</span> </h2>

In this section, I am going to talk about how to analyze with transfer learning for classification problem. To show this, let me use flower photos dataset which provided by google. 
First of all, let's import libraries.

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

<h2><span style="color:Purple">Loading the Dataset</span> </h2>

If you want you can directly load the dataset using get_file() method. `data_dir = tf.keras.utils.get_file('flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True`. By default, you can see this dataset in .keras file such as `/root/.keras/datasets/flower_photos` on your machine. Now that I am going to create a variable for directory of dataset. 

In [2]:
data_dir = "../input/flowers-recognition/flowers"

<h2><span style="color:Purple">Preprocessing the Dataset</span> </h2>

Now that I am going to define some hyperparameters such as pixel values and batch size.

In [3]:
pixels =224
BATCH_SIZE = 32
IMAGE_SIZE = (pixels, pixels)
NUM_CLASSES = 5

I am going to use ResNet model to analyze image classification. To use this model, you need to preprocess. Let me standardize image size to [223, 223, 3], and normalize pixel value tu a [0,1] range. First,let's create variable for hyperparameters.

In [4]:
datagen_kwargs = dict(rescale = 1./255, validation_split = .20)
dataflow_kwargs = dict(target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, interpolation = "bilinear")

I am going to split dataset into train and validation.  While validation dataset uses for cross validation, the other dataset uses for training model. 

In [5]:
# Generating batches of tensor image-data and creating validation dataset.
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory( data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

Found 860 images belonging to 5 classes.


Now, do the same for the training data generator. 

In [6]:
train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory( data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 3457 images belonging to 5 classes.


I am going to define class name. To do this, let me map flower class names.

In [7]:
labels_idx = (train_generator.class_indices)
idx_labels = dict((v,k) for k,v in labels_idx.items())
#Let's take a look how to map classes.
idx_labels

{0: 'daisy', 1: 'dandelion', 2: 'rose', 3: 'sunflower', 4: 'tulip'}

<h2><span style="color:Purple">Building the Model</span> </h2>

Datasets are ready to build the model. I am going to train using pretrain model. 

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/4", trainable=False),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'flower_class') 
])

model.build([None, 224, 224, 3])

<h2><span style="color:Purple">Compiling the Model</span> </h2>

After the model is built I am going to compile the model. To do this, let me specify the loss funciton and pick an optimizer.

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

Let's take a look summary of model.

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              42605504  
_________________________________________________________________
flower_class (Dense)         (None, 5)                 10245     
Total params: 42,615,749
Trainable params: 10,245
Non-trainable params: 42,605,504
_________________________________________________________________


<h2><span style="color:Purple">Training the Model</span> </h2>

Let's traing the model. First, let me determine the number of batches for training and cross-validation data.

In [11]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

Now that I am going to fit model.

In [12]:
model.fit(
    train_generator,
    epochs=5, 
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

Epoch 1/5
108/108 [==============================] - 55s 387ms/step - loss: 1.1933 - accuracy: 0.7062 - val_loss: 0.8538 - val_accuracy: 0.8281
Epoch 2/5
108/108 [==============================] - 22s 207ms/step - loss: 0.7529 - accuracy: 0.8702 - val_loss: 0.8119 - val_accuracy: 0.8401
Epoch 3/5
108/108 [==============================] - 22s 204ms/step - loss: 0.6159 - accuracy: 0.9276 - val_loss: 0.7251 - val_accuracy: 0.8762
Epoch 4/5
108/108 [==============================] - 22s 205ms/step - loss: 0.5700 - accuracy: 0.9480 - val_loss: 0.8214 - val_accuracy: 0.8329
Epoch 5/5
108/108 [==============================] - 22s 205ms/step - loss: 0.5962 - accuracy: 0.9396 - val_loss: 0.7267 - val_accuracy: 0.8642


<h2><span style="color:Purple">Predicting the Data</span> </h2>

You can use predict() method to score these validation images. Let's see predictions for first batch.

In [13]:
sample_test_images, ground_truth_labels = next(valid_generator)
prediction = model.predict(valid_generator)

In [14]:
len(prediction)

860

731 images and 5 corresponding classes in the cross-validation data print out on the screen. The highest probability in each row represents the prediction. I am going to find position where the highest probability occurs for each row.

In [15]:
predicted_idx = tf.math.argmax(prediction, axis = -1)

Let me display the result using print command.

In [16]:
print (predicted_idx)

tf.Tensor(
[0 0 0 0 0 0 3 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 3 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 4 1 1 3
 1 3 3 1 1 1 1 0 3 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 3 1 1 1 1 1 3 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 3 1 1 1 0 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 3 1 0 1 1 1 3 1 1 3 1 1 1 1 1 1 2 2 2 2 2 4 2 3
 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 4 2 2 2 4 2 2 2 2 2 2 2 4 2 2 2 2 2 2
 2 2 2 2 2 1 4 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 2 2 2 2 2 2 2 2 0 3 2 2
 2 2 2 2 2 2 2

<h2><span style="color:Purple">Evaluating the Model</span> </h2>

To evaluate the model, I am going to use confusion matrix, which compare model output with ground truth. To do this let me convert classes into Pandas Series structure and create variable, which include predictions.

In [17]:
y_actual = pd.Series(valid_generator.classes)
y_predicted = pd.Series(predicted_idx)

Then I am going to produce the matrix.

In [18]:
pd.crosstab(y_actual, y_predicted, rownames = ['Actual'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,3,4,All
Actual,,,,,,
0,140,5,1,5,1,152
1,9,184,1,15,1,210
2,1,5,132,9,9,156
3,1,6,4,133,2,146
4,4,10,20,8,154,196
All,155,210,158,170,167,860


Each row represents predicted value and each column display actual values. 
Now that let's take a look statistical report using sklearn library.

To see metrics my model, I going to create variables.

In [19]:
predicted_results = y_predicted
truth = y_actual

To evaluate the model, metrics are used. I am going to use sklearn library to see metrics. 

In [20]:
from sklearn.metrics import classification_report
report = classification_report(truth, predicted_results)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       152
           1       0.88      0.88      0.88       210
           2       0.84      0.85      0.84       156
           3       0.78      0.91      0.84       146
           4       0.92      0.79      0.85       196

    accuracy                           0.86       860
   macro avg       0.86      0.87      0.86       860
weighted avg       0.87      0.86      0.86       860



This results say that the model has good performance for classification problem. 

<h2><span style="color:Purple">Using the tf.keras.applications Module</span> </h2>

Keras is an API, which is a high level library. This module became a part of  the TensorFlow ecosystem in 2019. You can use Keras to fine tune your models. tf.keras.applications lets you determine which layers to retrain and which layers to untouched. Let's create a new base model using tf.keras.applications. 

In [21]:
base_model = tf.keras.applications.ResNet101V2(
    input_shape = (224, 224, 3), 
    include_top = False, 
    weights = 'imagenet')

171319296/171317808 [==============================] - 1s 0us/step


First, I specified input shape and then I set include_top to False so I can add my own Dense layer for the classification output. 
Now that I am going to build Sequential model based on base model and add GlobalAveragePooling2D layer. 

In [22]:
model2 = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax', name = 'flower_class')
])

Now that let me compile and fit the model.

In [23]:
model2.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
  loss=tf.keras.losses.CategoricalCrossentropy(
  from_logits=True, label_smoothing=0.1),
  metrics=['accuracy']
)
model2.fit(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

Epoch 1/5
108/108 [==============================] - 46s 339ms/step - loss: 1.0235 - accuracy: 0.6899 - val_loss: 0.9128 - val_accuracy: 0.7800
Epoch 2/5
108/108 [==============================] - 34s 316ms/step - loss: 0.5290 - accuracy: 0.9554 - val_loss: 0.6687 - val_accuracy: 0.8858
Epoch 3/5
108/108 [==============================] - 34s 317ms/step - loss: 0.4791 - accuracy: 0.9792 - val_loss: 0.6403 - val_accuracy: 0.9002
Epoch 4/5
108/108 [==============================] - 34s 318ms/step - loss: 0.4522 - accuracy: 0.9886 - val_loss: 0.6220 - val_accuracy: 0.9038
Epoch 5/5
108/108 [==============================] - 34s 318ms/step - loss: 0.4284 - accuracy: 0.9953 - val_loss: 0.6029 - val_accuracy: 0.9014


<h2><span style="color:Purple">Fine-Tuning Model</span> </h2>
You can fine tune your model using tf.keras.applications. Let's take a look number of layer in base model. 

In [24]:
print("Number of layers in the base model: ", len(base_model.layers))
base_model.trainable = True

Number of layers in the base model:  377


As you can see, there are 377 layers. I am going to take 370 layers as the starting layer for fine tuning. I want these layers not to be trained.

In [25]:
# Freezing all the layers before the 'fine_tune_at' layer
fine_tune_at = 300
for layer in base_model.layers[: fine_tune_at]:
    layer.trainable = False

Now that I am going to build the model again. 

In [26]:
model3 = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax', name = 'flower_class')
])

Let me compile the model.

In [27]:
model3.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1),
    metrics=['accuracy']
)

Let's take a look summary of the model3.

In [28]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101v2 (Functional)     (None, 7, 7, 2048)        42626560  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
flower_class (Dense)         (None, 5)                 10245     
Total params: 42,636,805
Trainable params: 19,189,253
Non-trainable params: 23,447,552
_________________________________________________________________


Let me train the model.

In [29]:
fine_tune_epochs = 2
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
model3.fit(
    train_generator,
    epochs=fine_tune_epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

Epoch 1/2
108/108 [==============================] - 33s 253ms/step - loss: 0.7754 - accuracy: 0.8275 - val_loss: 0.6068 - val_accuracy: 0.9087
Epoch 2/2
108/108 [==============================] - 24s 224ms/step - loss: 0.4337 - accuracy: 0.9963 - val_loss: 0.5915 - val_accuracy: 0.9147


My model has 0.996 accuracy value. This is a good value. That is all. In this tutorial, I showed how to use the pretrained model. To do pretrained model, there are two convenient ways: TensorFlow Hub and the tf.keras.applications module. You can easily train the your model according to your analysis using the pretrained models.

<h2><span style="color:Purple">Resources</span> </h2>

- [KC Tung, 2021, TensorFlow 2 Pocket Reference](https://www.amazon.com/TensorFlow-Pocket-Reference-Building-Deploying/dp/1492089184)
- [TensorFlow Tutorial](https://www.tensorflow.org/tutorials)

Don't forget to follow on Tirendaz Academy [YouTube-Tr](https://youtube.com/c/tirendazakademi), [YouTube-Eng](https://www.youtube.com/channel/UCFU9Go20p01kC64w-tmFORw), [Twitter](https://twitter.com/TirendazAcademy), [Medium](https://tirendazacademy.medium.com), [GitHub](https://github.com/TirendazAcademy) and [LinkedIn](https://www.linkedin.com/in/tirendaz-academy)